In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from jax import config

config.update("jax_enable_x64", True)
config.update("jax_platform_name", "cpu")

import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".8"

In [2]:
import jax.numpy as jnp
from jax import jit, vmap
import numpy as np
import matplotlib.pyplot as plt
import pickle
import h5py
import pandas as pd
import matplotlib as mpl
from scipy.ndimage import rotate

import jaxley as jx
from jaxley.channels import HH

In [4]:
# set directories
home_directory = os.path.expanduser("~")
data_dir = f'{home_directory}/GitRepos/jaxley_rgc/extracted_data'


In [3]:
def compute_jaxley_branch(roi_pos):
    '''
    '''
    min_dists = []
    min_comps = []
    for xyzr in cell.xyzr:
        dists = np.sum((roi_pos[:3] - xyzr[:, :3])**2, axis=1)
        min_dist = np.min(dists)
        argmin_dist = np.argmin(dists)
        if len(xyzr) > 1:
            comp_of_min = argmin_dist / (len(xyzr) - 1)
        else:
            comp_of_min = 0.5
        min_dists.append(min_dist)
        min_comps.append(comp_of_min)
        
    return np.argmin(min_dists), min_comps[np.argmin(min_dists)]

In [30]:
fnames = []
for (dirpath, dirnames, filenames) in os.walk(f"{data_dir}/morphology"):
    fnames.extend(filenames)

pkl_files = [f for f in os.listdir(data_dir) if f.startswith('cell') and f.endswith('.pkl')]
assert len(pkl_files) == 1

setup_df = pd.read_pickle(f"{data_dir}/{pkl_files[0]}")
recording_id = setup_df['recording_id'][0]
setup_df.head()


,Triggertimes_noise,Traces0_raw_noise,Tracetimes0_noise,roi_id,roi_pos,image_center,pixel_size,rec_id,cell_id,recording_id
0,"[16.892, 17.094, 17.294, 17.494, 17.694, 17.89...","[11109.35294117647, 11101.441176470587, 11098....","[0.022, 0.054, 0.086, 0.118, 0.15, 0.182, 0.21...",1,"[261.3911, 249.96454, 46.0]","[245.0, 250.0]",0.558594,d1,1,2020-07-08
1,"[16.892, 17.094, 17.294, 17.494, 17.694, 17.89...","[11000.916666666666, 11005.083333333334, 11011...","[0.011, 0.043, 0.075, 0.107, 0.139, 0.171, 0.2...",2,"[249.30276, 250.86816, 46.0]","[245.0, 250.0]",0.558594,d1,1,2020-07-08
2,"[16.892, 17.094, 17.294, 17.494, 17.694, 17.89...","[11002.235294117647, 10995.0, 10999.7058823529...","[0.013999999999999999, 0.046, 0.078, 0.11, 0.1...",3,"[246.67828, 253.23128, 46.0]","[245.0, 250.0]",0.558594,d1,1,2020-07-08
3,"[16.892, 17.094, 17.294, 17.494, 17.694, 17.89...","[11035.857142857143, 11024.714285714286, 11030...","[0.019999999999999997, 0.052, 0.08399999999999...",4,"[242.40187, 256.82776, 46.0]","[245.0, 250.0]",0.558594,d1,1,2020-07-08
4,"[16.892, 17.094, 17.294, 17.494, 17.694, 17.89...","[10985.333333333334, 11006.416666666666, 11004...","[0.009999999999999998, 0.041999999999999996, 0...",5,"[251.11696, 250.1089, 46.0]","[245.0, 250.0]",0.558594,d1,1,2020-07-08


In [45]:
write_dfs = []

for morph_full in fnames:
    cell_id = setup_df['cell_id'][0]
    df = setup_df
    cell = jx.read_swc(f'{data_dir}/morphology/{recording_id}/{morph_full}', nseg=4, max_branch_len=300.0, min_radius=1.0)
    
    
    
    for index, df_row in df.iterrows():
        roi_x,roi_y,roi_z = df_row['roi_pos']
        write_df = pd.DataFrame()
        jaxley_branch, jaxley_compartment = compute_jaxley_branch(df_row['roi_pos'])
        write_df["cell_id"] = [df_row["cell_id"]]
        write_df["rec_id"] = [df_row["rec_id"]]
        write_df["roi_id"] = [df_row["roi_id"]]
        write_df["roi_x"] = [roi_x]
        write_df["roi_y"] = [roi_y]
        write_df["roi_z"] = [roi_z]
        write_df["branch_ind"] = [int(jaxley_branch)]
        write_df["comp"] = [jaxley_compartment]

        write_dfs.append(write_df)
    
write_dfs = pd.concat(write_dfs).reset_index(drop=True)

/gpfs01/euler/User/ssuhai/GitRepos/jaxley/jaxley/utils/misc_utils.py:97: UserWarning: nseg is deprecated and will be removed in version 0.6.0.
  warnings.warn(msg + self._amend_msg)


In [47]:
write_dfs.head()

,cell_id,rec_id,roi_id,roi_x,roi_y,roi_z,branch_ind,comp
0,1,d1,1,261.391113,249.964539,46.0,28,0.051948
1,1,d1,2,249.302765,250.868164,46.0,26,0.052632
2,1,d1,3,246.678284,253.231277,46.0,26,0.074561
3,1,d1,4,242.401871,256.827759,46.0,26,0.092105
4,1,d1,5,251.116959,250.108902,46.0,26,0.043860


In [48]:
write_dfs.to_pickle(f"{data_dir}/recording_meta.pkl")